## Parameters

In [ ]:
# ------- PARAMETERS ------- #

RUNS = [17]               # run indexes

root = '/pnfs/pic.es/data/cta/LST/LST2/Data/ZFITS/' # path to all folders with the .zfits files

dir_files = 'csv_files/' # path (relative or absolute) to the .csv files that will be created

# -------------------------- #

# other parameters
only_subrun0  = False     # if we want to extract only the first subrun of a run
random_pixels = 3         # number of pixels we are extracting data, for the nanosecond scale analysis

## Import packages

In [ ]:
import numpy  as np
import pandas as pd

from traitlets.config import Config
from ctapipe.io       import EventSource
from IPython.display  import clear_output

import auxiliar as aux

LST_camera = aux.find_LST_num(root) # extracting the number of LST

# Extract data of PEDESTALS and its STDV

In [ ]:
# over all runs
for run in RUNS:

    # finding the run in the folder root
    date, subruns = aux.search(root,run)    

    # to open a second run we need to use the first subrun of the run and
    # extract the timestamp of the first event

    # first subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,
            },
        })  

    path   = root + date + '/'
    path   = path + 'LST-' + LST_camera + '.1.Run' + str(run).zfill(5) + '.' + str(0).zfill(4) + '.fits.fz'
    source = EventSource( input_url = path ,config = config, max_events=1)

    for i, ev in enumerate(source): 
        timestamp = int(ev.trigger.time.value)

    # following subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,

          "EventTimeCalculator": {
            "dragon_module_id": 133 ,
            "dragon_reference_counter": 0,
            "dragon_reference_time": timestamp,
          },
            },
        })
    
    # over all subruns
    if only_subrun0 == True:
        subruns = [0]
    for subrun in subruns:
        
        # directory
        path   = root + date + '/'
        path   = path+'LST-'+LST_camera+'.1.Run'+str(run).zfill(5)+'.'+str(subrun).zfill(4)+'.fits.fz'
        source = EventSource( input_url = path ,config = config)


        # arrays 
        mean = [[] for i in range(1855)]
        stdv = [[] for i in range(1855)]
        time =  []

        for i, ev in enumerate(source): 

            if i % 1000 == 0:
                print('Run '+str(run)+' Subrun '+str(subrun)+' - '+str(round(100 * i / 53000, 2))+'%')
                
            for j in range(1855):
                mean[j].append('%.2f' % np.mean(ev.r0.tel[1].waveform[0][j][5:]))   # pedestal
                stdv[j].append('%.2f' % np.std( ev.r0.tel[1].waveform[0][j][5:]))   # stdv
            time.append(ev.trigger.time.value)
            
        print('Run ' + str(run) + ' Subrun ' + str(subrun) + ' - 100%')
        print('\nCreating the dataframe...')
        
        #we create the matrix for the dataframe
        matrix = [time]  
        names  = ['Time']
        for i in range(len(mean)):
            matrix.append(mean[i])
            matrix.append(stdv[i])
            names.append('Pixel ' + str(i + 1) + ' mean')
            names.append('Pixel ' + str(i + 1) + ' stdv')

        del mean, stdv, time

        # creating the dataframe using pandas
        data         = pd.DataFrame(np.transpose(matrix))
        data.columns = names

        del matrix, names

        # csv arxive
        data.to_csv(dir_files + 'data_Run' + str(run) + '_Subrun' + str(subrun) + '.csv', index=False)

        # clear memory and console
        clear_output()
        del data
        
print('Finished with the pedestals file creation')

# Extract data of N random pixels (for nanosecond scale analysis)

In [ ]:
# over all runs
for run in RUNS:

    # finding the run in the folder root
    date, subruns = aux.search(root,run)    

    # to open a second run we need to use the first subrun of the run and
    # extract the timestamp of the first event

    # first subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,
            },
        })  

    path   = root + date + '/'
    path   = path + 'LST-' + LST_camera + '.1.Run' + str(run).zfill(5) + '.' + str(0).zfill(4) + '.fits.fz'
    source = EventSource( input_url = path ,config = config, max_events=1)

    for i, ev in enumerate(source): 
        timestamp = int(ev.trigger.time.value)

    # following subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,

          "EventTimeCalculator": {
            "dragon_module_id": 133 ,
            "dragon_reference_counter": 0,
            "dragon_reference_time": timestamp,
          },
            },
        })
    
    # N random pixels analysis for each run
    indexes = np.random.rand(random_pixels) * 1855
    indexes = np.array([int(indexes[i]) for i in range(len(indexes))])

    # over all subruns
    if only_subrun0 == True:
        subruns = [0]
    for subrun in subruns:
            
        # directory
        path   = root + date + '/'
        path   = path+'LST-'+LST_camera+'.1.Run'+str(run).zfill(5)+'.'+str(subrun).zfill(4)+'.fits.fz'
        source = EventSource(input_url=path, config=config)

        # arrays            
        charge = [[indexes[i] + 1] for i in range(len(indexes))]
        time   = [None]
                

        for i, ev in enumerate(source): 

            if i % 5500 == 0:
                print('Run '+str(run)+' Subrun '+str(subrun)+' - '+str(round(100 * i / 53000, 2))+'%')
            
            for i in range(len(indexes)):
                temp = list(ev.r0.tel[1].waveform[0][indexes[i]][5:])
                charge[i].append(temp)     


            time.append(ev.trigger.time.value)

        print('Run ' + str(run) + ' Subrun ' + str(subrun) + ' - 100%')
        print('\nCreating the dataframe...')

        #we create the matrix for the dataframe
        matrix = [time]  
        names  = ['Time']
        
        for i in range(len(charge)):
            matrix.append(charge[i])
            names.append('Pixel ' + str(indexes[i] + 1))

        del charge, time

        # creating the dataframe using pandas
        matrix       = np.array(matrix, dtype=object)
        data         = pd.DataFrame(np.transpose(matrix))
        data.columns = names

        del matrix, names

        #csv arxive
        data.to_csv(dir_files + 'data_Run' + str(run) + '_Subrun' + str(subrun) + '_nsScale.csv', index=False)

        # clear memory and console
        clear_output()
        del data
        
print('Finished with the nanosecond scale file creation')